# Loans Fundamentals

This notebook demonstrates how to access and use the functionalities of **Loans** which are part of our **QPS** module within LSEG Financial Analytics SDK.

In [1]:
from lseg_analytics.instruments.loans import search, load, delete

Search code:

In [2]:
# execute the search of loan templates
loan_templates = search()

print(loan_templates)

[{'type': 'Loan', 'id': '7caf85a5-7c3c-4ce8-b9f5-7db8918cb7a8', 'location': {'space': 'HOME', 'name': 'LoanResourceCreatd_For_Sdk_Notebooks'}, 'description': {'summary': '', 'tags': ['test']}}]


### Loading a specific loan template

User can load a specific loan template by either providing a unique template name or resource ID.

Mandatory arguments: 
- resource_id - mutually exclusive with name
- name - mutually exclusive with resource_id

In [3]:
# execute the search of loan templates using id
loaded_template = load(resource_id=loan_templates[0].id)

print(loaded_template)

<Loan space='HOME' name='LoanResourceCreatd_For_Sdk_Notebooks' 7caf85a5‥>


In [4]:
# execute the search of loan templates using name
loaded_template = load(name= loan_templates[0].location.name)

print(loaded_template)

<Loan space='HOME' name='LoanResourceCreatd_For_Sdk_Notebooks' 7caf85a5‥>


### Deleting a specific loan template

In [5]:
# Clone template to save original
cloned_template = loaded_template.clone()

# Save the cloned template to a space
cloned_template.save(name='template_for_deletion', space='HOME')

# Check that the loan with name 'template_for_deletion' exists
loan_templates = search()

print(loan_templates)

[{'type': 'Loan', 'id': '7caf85a5-7c3c-4ce8-b9f5-7db8918cb7a8', 'location': {'space': 'HOME', 'name': 'LoanResourceCreatd_For_Sdk_Notebooks'}, 'description': {'summary': '', 'tags': ['test']}}, {'type': 'Loan', 'id': '3a57e3ff-ec3c-4b60-99d1-4d60abb84e62', 'location': {'space': 'HOME', 'name': 'template_for_deletion'}, 'description': {'summary': '', 'tags': ['test']}}]


In [6]:
# Delete cloned template
delete(resource_id=cloned_template.id)


True

check that loan with name template_for_deletion is no longer available in the search

In [7]:
# The loan with name 'template_for_deletion' has been deleted
loan_templates = search()

print(loan_templates)

[{'type': 'Loan', 'id': '7caf85a5-7c3c-4ce8-b9f5-7db8918cb7a8', 'location': {'space': 'HOME', 'name': 'LoanResourceCreatd_For_Sdk_Notebooks'}, 'description': {'summary': '', 'tags': ['test']}}]


## Valuing loans

### Imports

Import the following necessary modules:

In [8]:
import json as js
from lseg_analytics.common import AdjustableDate, RelativeAdjustableDate, Date
from lseg_analytics.instruments.loans import LoanDefinitionInstrument, IrPricingParameters, LoanDefinition, ScheduleDefinition, OffsetDefinition, PrincipalDefinition, FloatingRateDefinition, BusinessDayAdjustmentDefinition
from lseg_analytics.instruments.loans import price, value

### Value a loan

### Create a loan for calculation

Create loan definition

In [9]:
business_day_adj = BusinessDayAdjustmentDefinition(calendars= [] , convention= "ModifiedFollowing")
start = AdjustableDate(date= "2021-03-02")
end =  RelativeAdjustableDate( reference_date= "StartDate", tenor= "10Y")
rate_definition = FloatingRateDefinition(index= "LSEG/EUR_ESTR_ON_FTSE")
interest_periods = ScheduleDefinition(business_day_adjustment= business_day_adj, start_date= start , end_date= end,frequency= "Annual", roll_convention= "Same")
offset_def = OffsetDefinition( business_day_adjustment= business_day_adj, direction= "Backward", reference_date= "PeriodEndDate", tenor= "0D" )
principal_definition = PrincipalDefinition( amount= 1000000, currency= "EUR")
loan_definition  = LoanDefinition(rate=rate_definition, interest_periods= interest_periods, payment_offset= offset_def, coupon_day_count= 'Dcb_Actual_360', accrual_day_count= 'Dcb_Actual_360', principal= principal_definition, payer= 'Party2', receiver= 'Party1')

In [10]:
loan_definition_Instrument = LoanDefinitionInstrument(definition=loan_definition)
pricing_parameters = IrPricingParameters( valuation_date= '2021-03-02')

### calculate a loan

The calculate function takes as input the list of loans to value, and some other optional parameters like pricing_preferences and return_market_data

calculate a loan using price formula

In [11]:
price_result = price(definitions= [loan_definition_Instrument], pricing_preferences= pricing_parameters)
print(price_result)

{'pricingPreferences': {'valuationDate': '2021-03-02', 'reportCurrency': 'EUR'}, 'analytics': [{'description': {'instrumentTag': '', 'instrumentDescription': 'Receive EUR Annual +0bp ESTR', 'startDate': '2021-03-02', 'endDate': '2031-03-03', 'tenor': '10Y'}, 'valuation': {'accrued': {'value': 0.0, 'percent': 0.0, 'dealCurrency': {'value': 0.0, 'currency': 'EUR'}, 'reportCurrency': {'value': 0.0, 'currency': 'EUR'}}, 'marketValue': {'value': -18572.358113432358, 'dealCurrency': {'value': -18572.358113432358, 'currency': 'EUR'}, 'reportCurrency': {'value': -18572.358113432358, 'currency': 'EUR'}}, 'cleanMarketValue': {'value': -18572.358113432358, 'dealCurrency': {'value': -18572.358113432358, 'currency': 'EUR'}, 'reportCurrency': {'value': -18572.358113432358, 'currency': 'EUR'}}}, 'risk': {'duration': {'value': 1.0}, 'modifiedDuration': {'value': -6.501023075545686e-05}, 'benchmarkHedgeNotional': {'value': 0.0, 'currency': 'EUR'}, 'annuity': {'value': 0.0, 'dealCurrency': {'value': 0.0

display price result as json

In [12]:
print(js.dumps(price_result.as_dict(), indent=4))

{
    "pricingPreferences": {
        "valuationDate": "2021-03-02",
        "reportCurrency": "EUR"
    },
    "analytics": [
        {
            "description": {
                "instrumentTag": "",
                "instrumentDescription": "Receive EUR Annual +0bp ESTR",
                "startDate": "2021-03-02",
                "endDate": "2031-03-03",
                "tenor": "10Y"
            },
            "valuation": {
                "accrued": {
                    "value": 0.0,
                    "percent": 0.0,
                    "dealCurrency": {
                        "value": 0.0,
                        "currency": "EUR"
                    },
                    "reportCurrency": {
                        "value": 0.0,
                        "currency": "EUR"
                    }
                },
                "marketValue": {
                    "value": -18572.358113432358,
                    "dealCurrency": {
                        "value": -18572.35811

calculate a loan using value formula to display loan's definitions in the output

In [13]:
value_result = value(definitions= [loan_definition_Instrument], pricing_preferences= pricing_parameters)
print(value_result)

{'definitions': [{'definition': {'rate': {'interestRateType': 'FloatingRate', 'index': 'LSEG/EUR_ESTR_ON_FTSE', 'leverage': 1.0}, 'interestPeriods': {'startDate': {'dateType': 'AdjustableDate', 'date': '2021-03-02'}, 'endDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '10Y', 'referenceDate': 'StartDate'}, 'frequency': 'Annual', 'businessDayAdjustment': {'calendars': [], 'convention': 'ModifiedFollowing'}, 'rollConvention': 'Same'}, 'paymentOffset': {'tenor': '0D', 'businessDayAdjustment': {'calendars': [], 'convention': 'ModifiedFollowing'}, 'referenceDate': 'PeriodEndDate', 'direction': 'Backward'}, 'couponDayCount': 'Dcb_Actual_360', 'accrualDayCount': 'Dcb_Actual_360', 'principal': {'currency': 'EUR', 'amount': 1000000.0}, 'payer': 'Party2', 'receiver': 'Party1'}}], 'pricingPreferences': {'valuationDate': '2021-03-02'}, 'analytics': [{'description': {'instrumentTag': '', 'instrumentDescription': 'Receive EUR Annual +0bp ESTR', 'startDate': '2021-03-02', 'endDate': '2031-03-03

display value result as json

In [14]:
print(js.dumps(value_result.as_dict(), indent=4))

{
    "definitions": [
        {
            "definition": {
                "rate": {
                    "interestRateType": "FloatingRate",
                    "index": "LSEG/EUR_ESTR_ON_FTSE",
                    "leverage": 1.0
                },
                "interestPeriods": {
                    "startDate": {
                        "dateType": "AdjustableDate",
                        "date": "2021-03-02"
                    },
                    "endDate": {
                        "dateType": "RelativeAdjustableDate",
                        "tenor": "10Y",
                        "referenceDate": "StartDate"
                    },
                    "frequency": "Annual",
                    "businessDayAdjustment": {
                        "calendars": [],
                        "convention": "ModifiedFollowing"
                    },
                    "rollConvention": "Same"
                },
                "paymentOffset": {
                    "tenor": "0D